In [1]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [2]:
dict_table = pickle.load(open('dict_table', 'rb'))
df_historical_data = pd.read_csv('clean_Uefa_euros_historical_data.csv')
df_fixture = pd.read_csv('clean_Uefa_euros_fixture.csv')

# Calculate Team Strength

In [3]:
df_historical_data.head(5)

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,Czechoslovakia,West Germany,1980,0,1,1
1,Belgium,West Germany,1980,1,2,3
2,Czechoslovakia,Italy,1980,1,1,2
3,Italy,Belgium,1980,0,0,0
4,England,Italy,1980,0,1,1


In [4]:
df_home = df_historical_data[['HomeTeam', 'HomeGoals', 'AwayGoals']]
df_away = df_historical_data[['AwayTeam', 'HomeGoals', 'AwayGoals']]

In [5]:
df_home = df_home.rename(columns={'HomeTeam': 'Team', 'HomeGoals': 'GoalsScored', 'AwayGoals': 'GoalsConceded'})
df_away = df_away.rename(columns={'AwayTeam': 'Team', 'HomeGoals': 'GoalsConceded', 'AwayGoals': 'GoalsScored'})

In [6]:
df_team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby('Team').mean()

df_team_strength

,GoalsScored,GoalsConceded
Team,,
Albania,0.333333,1.000000
Austria,0.700000,1.200000
Belgium,1.400000,1.250000
Bulgaria,0.666667,2.166667
CIS,0.333333,1.333333
Croatia,1.363636,1.272727
Czech Republic,1.241379,1.275862
Czechoslovakia,1.250000,1.000000
Denmark,1.322581,1.419355


# Function Predict Points

In [7]:
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        # goals_scored * goals_conceded
        lamb_home = df_team_strength.at[home,'GoalsScored'] * df_team_strength.at[away,'GoalsConceded']
        lamb_away = df_team_strength.at[away,'GoalsScored'] * df_team_strength.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 0)

# Predicting Euros

## Group Stage

In [8]:
dict_table['Group A']['Team'] = dict_table['Group A']['Team'].replace({'Germany (H)': 'Germany'})

In [9]:
df_fixture_group_36 = df_fixture[:36].copy()
df_fixture_knockout = df_fixture[36:44].copy()
df_fixture_quarter = df_fixture[44:48].copy()
df_fixture_semi = df_fixture[48:50].copy()
df_fixture_final = df_fixture[50:].copy()

In [ ]:
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group_36[df_fixture_group_36['home'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away

    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)

In [11]:
dict_table['Group A']

,Team,Pts
0,Germany,6.0
1,Switzerland,4.0
2,Hungary,4.0
3,Scotland,3.0


## Best Losers 

In [12]:
best_losers = []
for group in dict_table:
    third_place = dict_table[group].loc[2]
    best_losers.append(third_place)

df_best_loser = pd.DataFrame(best_losers)
df_best_loser = df_best_loser.sort_values('Pts', ascending=False, ignore_index=True)
df_best_loser = df_best_loser[:4]

In [13]:
df_best_loser

,Team,Pts
0,Hungary,4.0
1,Croatia,4.0
2,Poland,3.0
3,Ukraine,3.0


### Fixtures based on best loser

In [14]:
df_third_place_fixtures = pd.read_csv('third_place_fixtures.csv')

In [15]:
df_third_place_fixtures

,Group A,Group B,Group C,Group D,Group E,Group F,1B vs,1C vs,1E vs,1F vs
0,A,B,C,D,NaN,NaN,3A,3D,3B,3C
1,A,B,C,NaN,E,NaN,3A,3E,3B,3C
2,A,B,C,NaN,NaN,F,3A,3F,3B,3C
3,A,B,NaN,D,E,NaN,3D,3E,3A,3B
4,A,B,NaN,D,NaN,F,3D,3F,3A,3B
5,A,B,NaN,NaN,E,F,3E,3F,3B,3A
6,A,NaN,C,D,E,NaN,3E,3D,3C,3A
7,A,NaN,C,D,NaN,F,3F,3D,3C,3A
8,A,NaN,C,NaN,E,F,3E,3F,3C,3A
9,A,NaN,NaN,D,E,F,3E,3F,3D,3A


In [16]:
# Extract the groups that the best losers qualify from
qualifying_groups = []

# Iterate through each team in the provided list
for team in df_best_loser['Team']:
    # Check each group in dict_table to find the team
    for group, df in dict_table.items():
        if team in df['Team'].values:
            qualifying_groups.append(group)
            break 


qualifying_groups

['Group A', 'Group B', 'Group D', 'Group E']

In [17]:
qualifying_groups[3]

'Group E'

In [18]:
# Find the row in comb_table that matches the qualifying groups
for index, row in df_third_place_fixtures.iterrows():
    groups = row[:6].dropna().tolist()
    if all(group in qualifying_groups for groups in groups):
        third_place_combinations = row[6:].dropna().tolist()        
        break
print(third_place_combinations)

['3A', '3D', '3B', '3C']


In [19]:
third_place_combinations[2][1]

'B'

In [20]:
third_place_fixtures_dict = {'Winner Group B': f'3rd Group {third_place_combinations[0][1]}', 
                             'Winner Group C': f'3rd Group {third_place_combinations[1][1]}', 
                             'Winner Group E': f'3rd Group {third_place_combinations[2][1]}', 
                             'Winner Group F': f'3rd Group {third_place_combinations[3][1]}' }

third_place_fixtures_dict

{'Winner Group B': '3rd Group A',
 'Winner Group C': '3rd Group D',
 'Winner Group E': '3rd Group B',
 'Winner Group F': '3rd Group C'}

In [21]:
df_fixture_knockout

,home,score,away,year
36,Runner-up Group A,Match 38,Runner-up Group B,2024
37,Winner Group A,Match 37,Runner-up Group C,2024
38,Winner Group C,Match 40,3rd Group D/E/F,2024
39,Winner Group B,Match 39,3rd Group A/D/E/F,2024
40,Runner-up Group D,Match 42,Runner-up Group E,2024
41,Winner Group F,Match 41,3rd Group A/B/C,2024
42,Winner Group E,Match 43,3rd Group A/B/C/D,2024
43,Winner Group D,Match 44,Runner-up Group F,2024


In [22]:
def update_fixtures_with_third_place_teams(df_fixture_knockout, third_place_fixtures_dict, dict_table):
    # Iterate through the third place fixture dictionary
    for winner, third_placeholder in third_place_fixtures_dict.items():
        # Extract the group of the third place team from the placeholder
        third_group = third_placeholder.split()[-1]
        full_third_group = f'Group {third_group}'
        
        # Get the actual third place team from the corresponding group
        third_place_team = dict_table[full_third_group].loc[2, 'Team']
        
        # Replace the placeholder with the actual team in the fixtures
        df_fixture_knockout.loc[df_fixture_knockout['home'] == winner, 'away'] = third_place_team
    
    return df_fixture_knockout

     

In [23]:
updated_fixtures = update_fixtures_with_third_place_teams(df_fixture_knockout, third_place_fixtures_dict, dict_table)
updated_fixtures

,home,score,away,year
36,Runner-up Group A,Match 38,Runner-up Group B,2024
37,Winner Group A,Match 37,Runner-up Group C,2024
38,Winner Group C,Match 40,Poland,2024
39,Winner Group B,Match 39,Hungary,2024
40,Runner-up Group D,Match 42,Runner-up Group E,2024
41,Winner Group F,Match 41,Slovenia,2024
42,Winner Group E,Match 43,Croatia,2024
43,Winner Group D,Match 44,Runner-up Group F,2024


## Round of 16

In [24]:
df_fixture_knockout

,home,score,away,year
36,Runner-up Group A,Match 38,Runner-up Group B,2024
37,Winner Group A,Match 37,Runner-up Group C,2024
38,Winner Group C,Match 40,Poland,2024
39,Winner Group B,Match 39,Hungary,2024
40,Runner-up Group D,Match 42,Runner-up Group E,2024
41,Winner Group F,Match 41,Slovenia,2024
42,Winner Group E,Match 43,Croatia,2024
43,Winner Group D,Match 44,Runner-up Group F,2024


In [25]:
for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1, 'Team']
    df_fixture_knockout.replace({f'Winner {group}':group_winner,
                                 f'Runner-up {group}':runners_up}, inplace=True)

df_fixture_knockout['winner'] = '?'
df_fixture_knockout

,home,score,away,year,winner
36,Switzerland,Match 38,Italy,2024,?
37,Germany,Match 37,Denmark,2024,?
38,England,Match 40,Poland,2024,?
39,Spain,Match 39,Hungary,2024,?
40,France,Match 42,Romania,2024,?
41,Portugal,Match 41,Slovenia,2024,?
42,Belgium,Match 43,Croatia,2024,?
43,Netherlands,Match 44,Czech Republic,2024,?


In [26]:
def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_fixture_updated.loc[index, 'winner'] = winner
    return df_fixture_updated

In [27]:
get_winner(df_fixture_knockout)

,home,score,away,year,winner
36,Switzerland,Match 38,Italy,2024,Italy
37,Germany,Match 37,Denmark,2024,Germany
38,England,Match 40,Poland,2024,England
39,Spain,Match 39,Hungary,2024,Spain
40,France,Match 42,Romania,2024,France
41,Portugal,Match 41,Slovenia,2024,Portugal
42,Belgium,Match 43,Croatia,2024,Belgium
43,Netherlands,Match 44,Czech Republic,2024,Netherlands


## Quarter Finals 

In [28]:
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = df_fixture_round_1.loc[index, 'score']
        df_fixture_round_2.replace({f'Winner {match}':winner}, inplace=True)
    df_fixture_round_2['winner'] = '?'
    return df_fixture_round_2

In [29]:
update_table(df_fixture_knockout, df_fixture_quarter)

,home,score,away,year,winner
44,Spain,Match 45,Germany,2024,?
45,Portugal,Match 46,France,2024,?
46,England,Match 48,Italy,2024,?
47,Belgium,Match 47,Netherlands,2024,?


In [30]:
get_winner(df_fixture_quarter)

,home,score,away,year,winner
44,Spain,Match 45,Germany,2024,Spain
45,Portugal,Match 46,France,2024,France
46,England,Match 48,Italy,2024,Italy
47,Belgium,Match 47,Netherlands,2024,Netherlands


## Semi Finals

In [31]:
update_table(df_fixture_quarter, df_fixture_semi)

,home,score,away,year,winner
48,Spain,Match 49,France,2024,?
49,Netherlands,Match 50,Italy,2024,?


In [32]:
get_winner(df_fixture_semi)

,home,score,away,year,winner
48,Spain,Match 49,France,2024,Spain
49,Netherlands,Match 50,Italy,2024,Netherlands


## Final

In [33]:
update_table( df_fixture_semi, df_fixture_final)

,home,score,away,year,winner
50,Spain,Match 51,Netherlands,2024,?


In [34]:
get_winner(df_fixture_final)

,home,score,away,year,winner
50,Spain,Match 51,Netherlands,2024,Netherlands
